In [6]:
from models.GCN import GCN
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl
from trainer import train, test
from time import perf_counter

In [7]:
device = 'cpu'
activation = nn.ReLU()
epochs = 100
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4

In [8]:
data = PubmedGraphDataset()
g = data[0]
g = g.to(device)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[-1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


/Users/ferdy/Documents/hkust/fall2022/comp4222/EfficientGNN/efficient/lib/python3.9/site-packages/dgl/data/utils.py:288: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn('Property {} will be deprecated, please use {} instead.'.format(old, new))


In [9]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, n_hidden, n_classes, activation):
        super(GraphSAGE, self).__init__()
        self.conv1 = dgl.nn.SAGEConv(in_feats, n_hidden, 'mean')
        self.conv2 = dgl.nn.SAGEConv(n_hidden, n_classes, 'mean')
        self.activation = activation

    def forward(self, graph, features):
        h = self.conv1(graph, features)
        h = self.activation(h)
        h = self.conv2(graph, h)
        return h

In [10]:
model = GraphSAGE(in_feats, 16, n_classes, activation)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [11]:
train(model, g, features, labels, train_mask, val_mask, test_mask, epochs=epochs, loss_fn=loss_fn, optimizer=optimizer, lr=lr, weight_decay=weight_decay, device=device)

Epoch 00000 | Loss 1.1031 | Train: 0.8167, Val: 0.4980, Test: 0.5040
Epoch 00010 | Loss 0.2995 | Train: 0.9833, Val: 0.7460, Test: 0.7530
Epoch 00020 | Loss 0.0282 | Train: 1.0000, Val: 0.7420, Test: 0.7580
Epoch 00030 | Loss 0.0043 | Train: 1.0000, Val: 0.7480, Test: 0.7540
Epoch 00040 | Loss 0.0015 | Train: 1.0000, Val: 0.7480, Test: 0.7540
Epoch 00050 | Loss 0.0009 | Train: 1.0000, Val: 0.7480, Test: 0.7530
Epoch 00060 | Loss 0.0007 | Train: 1.0000, Val: 0.7420, Test: 0.7530
Epoch 00070 | Loss 0.0006 | Train: 1.0000, Val: 0.7440, Test: 0.7520
Epoch 00080 | Loss 0.0005 | Train: 1.0000, Val: 0.7440, Test: 0.7520
Epoch 00090 | Loss 0.0005 | Train: 1.0000, Val: 0.7420, Test: 0.7510
Training time: 3.0501s


In [12]:
train_acc, val_acc, test_acc = test(model, g, features, labels, train_mask, val_mask, test_mask, loss_fn=loss_fn, device=device)
print("Train accuracy: {:.4f}".format(train_acc))
print("Validation accuracy: {:.4f}".format(val_acc))
print("Test accuracy: {:.4f}".format(test_acc))

Train accuracy: 1.0000
Validation accuracy: 0.7440
Test accuracy: 0.7500
